## Employee Performance Analysis- Using CRISP DM Methodology

### INX Future Inc

### Business Understanding
### Problem Statement

#### INX Future Inc , (referred as INX ) , is one of the leading data analytics and automation solutions provider
with over 15 years of global business presence. INX is consistently rated as top 20 best employers past 5 years. Recent years, the employee performance indexes are not healthy and this is becoming a growing concerns among the top management.CEO, Mr. Brain, knows the issues but concerned to take any actions in penalizing non-performing employees as this would affect the employee morale of all the employees in general and may further reduce the performance

The following insights are expected from this project.

1. Department wise performances
2. Top 3 Important Factors effecting employee performance
3. A trained model which can predict the employee performance based on factors as inputs. This will be used to hire employees
4. Recommendations to improve the employee performance based on insights from analysis.

### Data Understanding
The data for training the model was collected from the company INX Future Inc. The data
collected included 1200 employee’s performance appraisal records, described by 28 parameters. the parameters show the different attributes of an employee based on which the prediction is to be made.

### Features present in the dataset

1-EmpNumber

2-Age

3-Gender

4-EducationBackground

5-MaritalStatus

6-EmpDepartment

7-EmpJobRole

8-BusinessTravelFrequency

9-DistanceFromHome

10-EmpEducationLevel

11-EmpEnvironmentSatisfaction

12-EmpHourlyRate

13-EmpJobInvolvement

14-EmpJobLevel

15-EmpJobSatisfaction

16-NumCompaniesWorked

17-OverTime

18-EmpLastSalaryHikePercent

19-EmpRelationshipSatisfaction

20-TotalWorkExperienceInYears

21-TrainingTimesLastYear

22-EmpWorkLifeBalance

23-ExperienceYearsAtThisCompany

24-ExperienceYearsInCurrentRole

25-YearsSinceLastPromotion

26-YearsWithCurrManager

27-Attrition

28-PerformanceRating


#### Importing essential libraries

In [ ]:
# 'Numpy' is used for mathematical operations on large, multi-dimensional arrays and matrices
import numpy as np

# 'Pandas' is used for data manipulation and analysis
import pandas as pd

# 'Seaborn' is based on matplotlib; used for plotting statistical graphics
import seaborn as sns

# 'Matplotlib' is a data visualization library for 2D and 3D plots, built on numpy
import matplotlib.pyplot as plt
%matplotlib inline


# suppress display of warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#### Loading the dataset

In [ ]:
# Importing the excel file
df = pd.read_excel("INX_Future_Inc_Employee_Performance_CDS_Project2_Data_V1.8.xls")

In [ ]:
df

### Exploring the dataset
### Exploratory Data Analysis

In [ ]:
# Returns number of rows and columns of the dataset
df.shape

In [ ]:
# Returns the first 5 number of rows
df.head(5)

In [ ]:
# Returns the last 5 number of rows
df.tail(5)

In [ ]:
# Returns an object with all of the column headers
df.columns

In [ ]:
# Returns basic information on all columns
df.info()

In [ ]:
# Returns basic statistics on numeric columns
df.describe().T

In [ ]:
# Returns different datatypes for each columns
df.dtypes

In [ ]:
# Returns total number of missing values for each columns
df.isnull().sum()

### Data Visualization

In [ ]:
# New Dataframe is created to analyze department wise performance.
dept = df.iloc[:,[5,27]].copy()
dept_per = dept.copy()

In [ ]:
# Finding the mean performance of all the departments and plotting bar graph using seaborn.
dept_per.groupby(by='EmpDepartment')['PerformanceRating'].mean()

In [ ]:
plt.figure(figsize=(10,4.5))
sns.barplot(dept_per['EmpDepartment'],dept_per['PerformanceRating'])

In [ ]:
# Analyze all department separately
dept_per.groupby(by='EmpDepartment')['PerformanceRating'].value_counts()

In [ ]:
# Creating new Dataframe to analyze each department separately
department = pd.get_dummies(dept_per['EmpDepartment'])
performance = pd.DataFrame(dept_per['PerformanceRating'])
dept_rating = pd.concat([department,performance],axis=1)

In [ ]:
# Plotting separate bar graph for performance of each department using seaborn
plt.figure(figsize=(15,10))
plt.subplot(2,3,1)
sns.barplot(dept_rating['PerformanceRating'],dept_rating['Sales'])
plt.subplot(2,3,2)
sns.barplot(dept_rating['PerformanceRating'],dept_rating['Development'])
plt.subplot(2,3,3)
sns.barplot(dept_rating['PerformanceRating'],dept_rating['Research & Development'])
plt.subplot(2,3,4)
sns.barplot(dept_rating['PerformanceRating'],dept_rating['Human Resources'])
plt.subplot(2,3,5)
sns.barplot(dept_rating['PerformanceRating'],dept_rating['Finance'])
plt.subplot(2,3,6)
sns.barplot(dept_rating['PerformanceRating'],dept_rating['Data Science'])
plt.show()

### Data Cleaning
The raw data we have is completely clean. We checked for missing values but data contained no missing
values. To get proper prediction we need to do label encoding for categorical values

### Correlation Plot

In [ ]:
plt.figure(figsize=(20,18))
sns.heatmap(df.corr(), cmap='YlGnBu',annot=True)
plt.show()

In [ ]:
sns.pairplot(df, diag_kind='kde')

### Data Encoding/Data wrangling

In [ ]:
# Encoding all the ordinal columns and creating a dummy variable for them to see if there are any effects on Performance Rating
enc = LabelEncoder()
for i in (2,3,4,5,6,7,16,26):
    df.iloc[:,i] = enc.fit_transform(df.iloc[:,i])
df.head()

### Feature Selection

In [ ]:
# Finding out the correlation coeffecient to find out which predictors are significant.
df.corr()

There are a lot of columns in the predictor variable. So, the correlation coeffecient is calculated to see which of them are important and these are then used for training methods. From there, we also get the top factors which affect performance. We can see that the most important features selectd were Department, Job Role, Environment Satisfaction, Last Salary Hike Percent, Work Life Balance, Experience Years At This Company, Experience Years In Current Role, Years Since Last Promotion, Years With Current Manager. These were selected because their correlation coeffecient with Performance Rating was more than 0.1. Standardization and Label Encoding was also used for feature transformation.

A separate analysis considering all the predictors was carried out but it resulted in decreasing the accuracy. Similarly, Principal Component Analysis also reduces the accuracy.
Top 3 factors which affect the employee performance are-
1. Employee EnvironmentSatisfaction
2. Employee Last Salary Hike Percent
3. Years Since Last Promotion

In [ ]:
# Dropping the first columns as it is of no use for analysis.
df.drop(['EmpNumber'],inplace=True,axis=1)

In [ ]:
# Selected only the important columns
y = df.PerformanceRating
X = df.iloc[:,[4,5,9,16,20,21,22,23,24]] # Taking only variables with correlation coeffecient greater than 0.1
X.head()

### Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)
X_train.shape, X_test.shape,y_train.shape, y_test.shape

### Feature Scaling

In [ ]:
# Standardization technique is used
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

### Modeling
Used algorithms are Logistic Regression, Support Vector Machine, Decision Tree, Random Forest,Naive Bayes Bernoulli,K-Nearest Neighbor and Artificial Neural Network

### 1. Logistic Regression

In [ ]:
# Training the model
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression()
model_lr.fit(X_train,y_train)

In [ ]:
# Predicting the model
y_predict_lr = model_lr.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test,y_predict_lr))
print(classification_report(y_test,y_predict_lr))

In [ ]:
confusion_matrix(y_test,y_predict_lr)

In [ ]:
lr= accuracy_score(y_test,y_predict_lr)

### 2. Support Vector Machine

In [ ]:
# Training the model
from sklearn.svm import SVC
rbf_svc = SVC(kernel='rbf', C=100, random_state=10).fit(X_train,y_train)

In [ ]:
# Predicting the model
y_predict_svm = rbf_svc.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test,y_predict_svm))
print(classification_report(y_test,y_predict_svm))

In [ ]:
confusion_matrix(y_test,y_predict_svm)

In [ ]:
svm= accuracy_score(y_test,y_predict_svm)

### 3. Decision Tree with GridSearchCV

In [ ]:
# Training the model
from sklearn.tree import DecisionTreeClassifier

classifier_dt=DecisionTreeClassifier(random_state=42,splitter='best')
parameters=[{'min_samples_split':[2,3,4,5],'criterion':['gini']},{'min_samples_split':[2,3,4,5],'criterion':['entropy']}]

model_dt_grid=GridSearchCV(estimator=classifier_dt, param_grid=parameters, scoring='accuracy',cv=10)
model_dt_grid.fit(X_train,y_train)

In [ ]:
model_dt_grid.best_params_

In [ ]:
# Predicting the model
y_predict_dt = model_dt_grid.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test,y_predict_dt))
print(classification_report(y_test,y_predict_dt))

In [ ]:
confusion_matrix(y_test,y_predict_dt)

In [ ]:
dt_grid= accuracy_score(y_test,y_predict_dt)

### 4. Random Forest with GridSearchCV

In [ ]:
# Training the model
from sklearn.ensemble import RandomForestClassifier

classifier_rfg=RandomForestClassifier(random_state=33,n_estimators=23)
parameters=[{'min_samples_split':[2,3,4,5],'criterion':['gini','entropy'],'min_samples_leaf':[1,2,3]}]

model_rf_grid=GridSearchCV(estimator=classifier_rfg, param_grid=parameters, scoring='accuracy',cv=10)
model_rf_grid.fit(X_train,y_train)

In [ ]:
model_rf_grid.best_params_

In [ ]:
# Predicting the model
y_predict_rf = model_rf_grid.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test,y_predict_rf))
print(classification_report(y_test,y_predict_rf))

In [ ]:
confusion_matrix(y_test,y_predict_rf)

In [ ]:
rf_grid= accuracy_score(y_test,y_predict_rf)

### 5. Naive Bayes Bernoulli

In [ ]:
# Training the model
from sklearn.naive_bayes import BernoulliNB
model_nb = BernoulliNB()
model_nb.fit(X_train,y_train)

In [ ]:
# Predicting the model
y_predict_nb = model_nb.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test,y_predict_nb))
print(classification_report(y_test,y_predict_nb))

In [ ]:
confusion_matrix(y_test,y_predict_nb)

In [ ]:
nb= accuracy_score(y_test,y_predict_nb)

### 6. K-Nearest Neighbor

In [ ]:
# Training the model
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=10,metric='euclidean') # Maximum accuracy for n=10
model_knn.fit(X_train,y_train)

In [ ]:
# Predicting the model
y_predict_knn = model_knn.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test,y_predict_knn))
print(classification_report(y_test,y_predict_knn))

In [ ]:
confusion_matrix(y_test,y_predict_knn)

In [ ]:
knn= accuracy_score(y_test,y_predict_knn)

### 7. Artificial Neural Network

In [ ]:
# Training the model
from sklearn.neural_network import MLPClassifier
model_mlp = MLPClassifier(hidden_layer_sizes=(100,100,100),batch_size=10,learning_rate_init=0.01,max_iter=2000,random_state=10)
model_mlp.fit(X_train,y_train)

In [ ]:
# Predicting the model
y_predict_mlp = model_mlp.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test,y_predict_mlp))
print(classification_report(y_test,y_predict_mlp))

In [ ]:
confusion_matrix(y_test,y_predict_mlp)

In [ ]:
ANN= accuracy_score(y_test,y_predict_mlp)

### Model Performance
Accuracy score comparison chart

In [ ]:
Accuracy_Score = [lr,svm,dt_grid,rf_grid,nb,knn,ANN]
Models = ['Logistic Regression', 'Support Vector Machine' , 'Decision Tree with GridSearchCV', 'Random Forest with GridSearchCV',
          'Naive Bayes Bernoulli', 'K-Nearest Neighbor', 'Artificial Neural Network']

In [ ]:
sns.barplot(Accuracy_Score, Models, color="xkcd:baby poop green")
plt.xlabel('Accuracy Score')
plt.title('Accuracy Score')
plt.show()

### Analysis
The data is supervised and categorical. The predictor variables are ordinal and a few among them are nominal. The target variable 'Performance Rating' is ordinal.

To analyze the data, various data processing techniques like Label Encoding and Standardization is used.

Correlation Coeffecient is used to interpret the relationship between variables.

The most important features selected are Department, Job Role, Environment Satisfaction, Last Salary Hike Percent, Work Life Balance, Experience Years At This Company, Experience Years In Current Role, Years Since Last Promotion, Years With Current Manager.

For training the data and predicting the target, algorithms used are Logistic Regression, Support Vector Machine, Decision Tree, Random Forest, Naive Bayes, K-Nearest Neighbor and Artificial Neural Network.

A separate analysis of Department wise Performance is done as well.

### Results
Random Forest with GridSearchCV gives 94% accuracy.

The features that are positively correlated are Environment Satisfaction, Last Salary Hike Percent & Worklife Balance. This means that if these factors increases, Performance Rating will increase. On the other hand, the features that are negatively correlated are Years Since Last Promotion, Experience Years at this Company, Experience years in Current Role & Years with Current Manager. This means that if these factors increases, Performance Rating will go down.

The top 3 features effecting employee performances are:
Employee Environment Satisfaction
Employee Last Salary Hike Percent
Years since last promotion

### Analysis and Insights
It is observed that the maximum accuracy is obtained when we used Random Forest with GridSearchCV which is 94%. Decision tree and Artificial neural networks also yielded an accuracy of 92.5%.

### Recommendation
We can conclude that the company should provide a better environment as it increases the performance drastically. The company should increase the salary of the employee from time to time and help them maintain a worklife balance. On the other hand, shuffling the manager from time to time will also affect performance.